# Step 3: Performing a linear regression to predict ambulance calls

**Goal of the step:** Creating a statistical model that includes both geographical information and demographic data to predict ambulance calls.

**Step overview:**
1. Loading of the dataframe
2. Fitting of the first model
3. Inclusion of the geographical feature
4. Results

In [ ]:
import pandas as pd
import osmnx
import geopandas as gpd
import rioxarray
import xarray
import datashader
import contextily as cx
from shapely import geometry
from shapely import wkt
import matplotlib.pyplot as plt
import seaborn
from pysal.viz import splot
from splot.esda import plot_moran
import contextily
from pysal.explore import esda
from pysal.lib import weights
from numpy.random import seed
import numpy as np
import os
import statsmodels.formula.api as sm
from pysal.model import spreg
import statsmodels.formula.api as sm

## 1. Loading of the dataframe

In [ ]:
df = pd.read_csv("data/df.csv")

In [ ]:
# Setting of the geometry type

df['geometry'] = gpd.GeoSeries.from_wkt(df['geometry'])
df = gpd.GeoDataFrame(df, geometry='geometry')

## 2. Fitting the first linear model

In [ ]:
# Fit OLS model
m1 = spreg.OLS(
    # Dependent variable
    df[['sqrt_calls']].values, 
    # Independent variables
    df[['pop', 'per_for', 'dist_road',
       'unemp','p0_14',
         'p65_PL']].values,
    # Dependent variable name
    name_y='n_calls', 
    # Independent variable name
    name_x=['pop', 'per_for', 'dist_road',
       'unemp','p0_14',
         'p65_PL']
)

In [ ]:
print(m1.summary)

In this first model, population, percentage of foreigners, distance from the closest provincial/national road, percentage of unemployed, percentage of inhabitants who are less than 15 years old, and percentage of inhabitants who are more than 65 years old are used to predict the square root of the number of ambulance calls.
,,
Without considering the geographical feature and the median income, the constant term and all the variables used *(apart from unemployment)* appear to be statistically significant *(p-value < 5%)* in predicting ambulance calls.

The adjusted R squared of the model is equal to 57%, meaning that the independent variables used explain more than half the variance of emergency calls.

The level of multicollinearity among the independent variables is 10, considered a limit value for the acceptance of the model.

This model shows that population density, percentage of foreigners, distance from the closest provincial/national road, and percentage of inhabitants who are 65 years old or more are positively correlated with the number of ambulance calls.

On the other end, an increasing percentage of inhabitants who are younger than 15 years old is shown to be negatively correlated with the number of emergency calls.

In [ ]:
knn = weights.KNN.from_dataframe(df, k=1)

In [ ]:
lag_residual = weights.spatial_lag.lag_spatial(knn, m1.u)
ax = seaborn.regplot(
    m1.u.flatten(), 
    lag_residual.flatten(), 
    line_kws=dict(color='orangered'),
    ci=None
)
ax.set_xlabel('Model Residuals - $u$')
ax.set_ylabel('Spatial Lag of Model Residuals - $W u$');

By plotting the prediction error of a cell, and the prediction error of the closest other cell, we see that prediction errors tend to cluster, meaning that if the model overpredicts the number of calls in one cell, it will likely overpredict also in the closest cell.

This behaviour of the residuals is critical, as it expresses an underlying geographical relationship that if considered in the model can improve its prediction power.

## 3. Including the geographical feature

Now a new linear model will be created to explicitely include geospatial information in the model.

Since the model is missing some explanatory variables (only explains 57% of the variance), it might miss some important characteristics that influence the number of ambulance calls.
Some of these characteristics are likely to vary over space, therefore by including the neighbourhoods as categorical variables, the model will be able to control some of these unobserved features.

In [ ]:
# New regression model with neighbourhood and median income as categorical variables

f = 'sqrt_calls ~ ' + ' + '.join(['pop', 'per_for', 'dist_road',
       'unemp','p0_14',
         'p65_PL'])  +  ' + BU_CODE - 1' + ' + med_inc - 1' 
print(f)

In [ ]:
m2 = sm.ols(f, data=df).fit()

In [ ]:
results_as_html = m2.summary2().tables[1]

In [ ]:
results_as_html 

## 4. Results

In [ ]:
# Subset of the features only containing the statistically significant attributes

results_as_html[results_as_html['P>|t|']<=0.05]

In [ ]:
m2.summary2()

The prediction power of the model increased, with the adjusted R squared up to 67.3% and R squared up to 80%.

After including these two categorical variables, the distance from the closest provincial/national road is no longer significant, this could be because in the previous model this variable was also explaining some geographical characteristics (e.g. how people cluster in a city based on their income level) that are now being controlled by the new model, and the distance from provincia/national roads itselft does not exlain the number of ambulance calls.
The inclusion of new variables had the opposite effect on the significante of unemployment, which is now down to 18% from 47%, this could be explained with the same reasoning presented above, but this time the effect itself of unemployment is significant at a significance level of 20%.

From these results, it appears that emergency calls have indeed a geographical component that is not captured by the demographic features used in the model.

When including categorical variables, the p-value specifies wether the coefficient estimated for a model is significantly different from the one of the reference level.
This model points out that for some neighborhoods the coefficient is not significantly different from the one of the reference level, while for other neighbourhoods it is significantly different *(reference level + coefficient of the categorical variable)*.
At the same time the levels of median income are not statistically significant, indicating that the different levels of income do not influence the number of ambulance calls.
At the same time, it is important to note that a low median income has a positive coefficient and would be significant with a significance level of 10%.

Further discussion and comments about the results are presented in the report.